## [Pandas Reference](https://pandas.pydata.org/pandas-docs/stable/index.html)
1. [Find the unique values in a column and then sort them](https://stackoverflow.com/questions/32072076/find-the-unique-values-in-a-column-and-then-sort-them)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
print('pandas version: {}'.format(pd.__version__))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))

In [ ]:
expenses = pd.read_csv('Expenses.csv')

cols = expenses.columns.tolist()
print(cols)
uniqueAccounts = expenses['Account'].unique()
uniqueAccounts.sort()
print(uniqueAccounts)
uniqueCategories = expenses['Category'].unique()
uniqueCategories.sort()
print(uniqueCategories)
expenses[expenses['Account'].isin(['Hilton Honors Card', 'chunyenwang-8046'])]

In [ ]:
expensesdata.head()